In [1]:
import oci

In [2]:
compartment_id=
namespace_name=
bucket_name="uoft-mlops-w23-labs"


source_uri = "oci://{}@{}/WordCount/input/pg2701.txt".format(bucket_name,namespace_name)
model_uri = "oci://{}@{}/WordCount/model/WordCount.py".format(bucket_name,namespace_name)
config_uri = "oci://{}@{}/config/config.txt".format(bucket_name,namespace_name)
bucket_uri = "oci://{}@{}".format(bucket_name,namespace_name)
destination_uri = "oci://{}@{}/WordCount/output".format(bucket_name,namespace_name)

key_file=
fingerprint=
region= 
user= 

In [3]:
# https://docs.oracle.com/en-us/iaas/Content/API/Concepts/apisigningkey.htm
config = {
    "user": user,
    "key_file": key_file,
    "fingerprint": fingerprint,
    "tenancy": compartment_id,
    "region": region
}


from oci.config import validate_config
validate_config(config)

header="[DEFAULT]\n"
with open("ocikey/config.txt", "w") as file:
    file.write(header)
    for key, value in config.items():
        file.write(f"{key}={value}\n")


In [4]:
from oci.config import from_file 
# https://docs.cloud.oracle.com/en-us/iaas/Content/API/Concepts/sdkconfig.htm#SDK_and_CLI_Configuration_File
config = from_file(file_location="ocikey/config.txt")

In [5]:
# Initialize service client with default config file
object_storage_client = oci.object_storage.ObjectStorageClient(config)

In [ ]:
# get_bucket_response = object_storage_client.get_bucket(
#     namespace_name="axf2ytdzdvoh",
#     bucket_name="bucket-20230201-1739",
#     #if_match="EXAMPLE-ifMatch-Value",
#     #if_none_match="EXAMPLE-ifNoneMatch-Value",
#     #opc_client_request_id="ocid1.test.oc1..<unique_ID>EXAMPLE-opcClientRequestId-Value",
#     #fields=["autoTiering"]
# )
# get_bucket_response

In [ ]:
# Send the request to service, some parameters are not required, see API
# doc for more info
create_bucket_details=oci.object_storage.models.CreateBucketDetails(
        name=bucket_name,
        compartment_id=compartment_id
)
create_bucket_response = object_storage_client.create_bucket(
    namespace_name=namespace_name,
    create_bucket_details=create_bucket_details
)

# Get the data from response
print(create_bucket_response.data)

In [6]:
with open('pg2701.txt', 'rb') as file:
    object_storage_client.put_object(namespace_name,bucket_name, 'WordCount/input/pg2701.txt', file)

In [7]:
with open('WordCount.py', 'rb') as file:
    object_storage_client.put_object(namespace_name,bucket_name, 'WordCount/model/WordCount.py', file)

In [8]:
with open('ocikey/config.txt', 'rb') as file:
    object_storage_client.put_object(namespace_name,bucket_name, 'config/config.txt', file)

In [9]:
data_flow_client = oci.data_flow.DataFlowClient(config)

In [10]:
application_details=oci.data_flow.models.CreateApplicationDetails(
compartment_id=compartment_id,
display_name="Word_Count_Application",
driver_shape="VM.Standard2.1",
executor_shape="VM.Standard2.1",
language="PYTHON",
num_executors=1,
spark_version="3.2.1",
file_uri=model_uri,
logs_bucket_uri=bucket_uri
)

In [11]:
application=data_flow_client.create_application(
    create_application_details=application_details
)
if application.status !=200:
    print("Failed to create application")
else:
    print("application is created")

application is created


In [12]:
run_details = oci.data_flow.models.CreateRunDetails(
compartment_id=application.data.compartment_id,
application_id=application.data.id,
display_name=application.data.display_name
)

In [13]:
run=data_flow_client.create_run(
    create_run_details=run_details
)
if run.status !=200:
    print("Failed to create run")
else:
    print("run is created")

run is created


In [20]:
objects = object_storage_client.list_objects(namespace_name, bucket_name, prefix="WordCount/output/word_counts").data.objects
for obj in objects:
    print(obj.name)

WordCount/output/word_counts/
WordCount/output/word_counts/_SUCCESS
WordCount/output/word_counts/part-00000
WordCount/output/word_counts/part-00001


In [21]:
response1 = object_storage_client.get_object(namespace_name, bucket_name, 'WordCount/output/word_counts/part-00000')
response2 = object_storage_client.get_object(namespace_name, bucket_name, 'WordCount/output/word_counts/part-00001')

In [39]:
response1.data.content

b"('project', 91)\n('gutenberg', 94)\n('ebook', 10)\n('of', 6732)\n('moby', 89)\n('melville', 4)\n('this', 1443)\n('is', 1751)\n('use', 49)\n('anyone', 6)\n('anywhere', 16)\n('at', 1335)\n('no', 596)\n('restrictions', 2)\n('whatsoever', 7)\n('may', 255)\n('give', 90)\n('away', 186)\n('online', 4)\n('www', 6)\n('org', 13)\n('title', 8)\n('last', 278)\n('january', 1)\n('posting', 1)\n('june', 3)\n('language', 7)\n('start', 37)\n('produced', 9)\n('transcriber', 1)\n('s', 1816)\n('notes', 4)\n('combination', 2)\n('now', 786)\n('defunct', 2)\n('eris', 1)\n('virginia', 6)\n('version', 6)\n('are', 619)\n('university', 1)\n('adelaide', 1)\n('library', 3)\n('preserving', 4)\n('resulting', 3)\n('was', 1646)\n('compared', 9)\n('public', 15)\n('domain', 9)\n('in', 4231)\n('we', 470)\n('l', 7)\n('symbol', 12)\n('late', 30)\n('consumptive', 1)\n('usher', 2)\n('grammar', 2)\n('school', 10)\n('pale', 19)\n('heart', 91)\n('i', 2127)\n('he', 1899)\n('ever', 206)\n('dusting', 2)\n('his', 2530)\n('lexicon